# Reddit 搜索

在本笔记中，我们将学习 Reddit 搜索工具的工作原理。
首先，请确保您已通过以下命令安装了 praw：

In [ ]:
%pip install --upgrade --quiet  praw

然后，您需要设置正确的 API 密钥和环境变量。您需要创建一个 Reddit 用户账户并获取凭据。

首先，请访问 [https://www.reddit.com](https://www.reddit.com) 注册一个 Reddit 用户账户。

接着，访问 [https://www.reddit.com/prefs/apps](https://www.reddit.com/prefs/apps) 创建一个应用以获取您的凭据。

在创建应用后，您将获得 `client_id` 和 `client_secret`。现在，您可以将这些字符串粘贴到 `client_id` 和 `client_secret` 变量中。

**注意：** `user_agent` 可以是任意字符串。

In [ ]:
client_id = ""
client_secret = ""
user_agent = ""

In [ ]:
from langchain_community.tools.reddit_search.tool import RedditSearchRun
from langchain_community.utilities.reddit_search import RedditSearchAPIWrapper

search = RedditSearchRun(
    api_wrapper=RedditSearchAPIWrapper(
        reddit_client_id=client_id,
        reddit_client_secret=client_secret,
        reddit_user_agent=user_agent,
    )
)

然后，您可以设置查询，例如，您要查询哪个 subreddit，希望返回多少帖子，以及您希望结果如何排序等。

In [ ]:
from langchain_community.tools.reddit_search.tool import RedditSearchSchema

search_params = RedditSearchSchema(
    query="beginner", sort="new", time_filter="week", subreddit="python", limit="2"
)

最终运行搜索并获取结果

In [ ]:
result = search.run(tool_input=search_params.dict())

In [ ]:
print(result)

下面是打印结果的示例。
注意：根据 subreddit 中的最新帖子，您可能会得到不同的输出，但格式应该相似。

>在 r/python 中搜索到 2 篇帖子：
>帖子标题：“在 Visual Studio Code 中设置 Github Copilot”
>用户：Feisty-Recording-715
>子版块：r/Python：
>                     正文：🛠️ 本教程非常适合希望加强对版本控制理解的初学者，或者希望快速参考 GitHub 在 Visual Studio Code 中设置的经验开发者。
>
>🎓 观看完本视频后，您将掌握自信地管理代码库、与他人协作以及为 GitHub 上的开源项目做出贡献的技能。
>
>
>视频链接：https://youtu.be/IdT1BhrSfdo?si=mV7xVpiyuhlD8Zrw
>
>欢迎您提出意见
>                     帖子网址：https://www.reddit.com/r/Python/comments/1823wr7/setup_github_copilot_in_visual_studio_code/
>                     帖子类别：N/A。
>                     得分：0
>
>帖子标题：“使用 pygame 和 PySide6 制作的中国跳棋游戏，支持自定义机器人”
>用户：HenryChess
>子版块：r/Python：
>                     正文：GitHub 链接：https://github.com/henrychess/pygame-chinese-checkers
>
>我不确定这是否算初级或中级。我认为我仍然处于初级阶段，所以我将其标记为初级。
>
>这是一个供 2-3 名玩家玩的中国跳棋（也称为 Sternhalma）游戏。我编写的机器人很容易被击败，因为它们主要用于调试代码的游戏逻辑部分。但是，您可以编写自己的自定义机器人。 GitHub 页面上有一个指南。
>                     帖子网址：https://www.reddit.com/r/Python/comments/181xq0u/a_chinese_checkers_game_made_with_pygame_and/
>                     帖子类别：N/A。
 >                    得分：1

## 使用带代理链的工具

Reddit 搜索功能也作为多输入工具提供。在本例中，我们改编了[文档中的现有代码](https://python.langchain.com/v0.1/docs/modules/memory/agent_with_memory/)，并使用 ChatOpenAI 创建了一个带内存的代理链。该代理链能够从 Reddit 获取信息，并利用这些帖子响应后续的输入。

要运行此示例，请添加您的 Reddit API 访问信息，并从[OpenAI API](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key) 获取一个 OpenAI 密钥。

In [ ]:
# Adapted code from /docs/modules/agents/how_to/sharedmemory_for_tools

from langchain.agents import AgentExecutor, StructuredChatAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain_community.tools.reddit_search.tool import RedditSearchRun
from langchain_community.utilities.reddit_search import RedditSearchAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

# Provide keys for Reddit
client_id = ""
client_secret = ""
user_agent = ""
# Provide key for OpenAI
openai_api_key = ""

template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

tools = [
    RedditSearchRun(
        api_wrapper=RedditSearchAPIWrapper(
            reddit_client_id=client_id,
            reddit_client_secret=client_secret,
            reddit_user_agent=user_agent,
        )
    )
]

prompt = StructuredChatAgent.create_prompt(
    prefix=prefix,
    tools=tools,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = StructuredChatAgent(llm_chain=llm_chain, verbose=True, tools=tools)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, verbose=True, memory=memory, tools=tools
)

# Answering the first prompt requires usage of the Reddit search tool.
agent_chain.run(input="What is the newest post on r/langchain for the week?")
# Answering the subsequent prompt uses memory.
agent_chain.run(input="Who is the author of the post?")